In [2]:
import pymysql
import pandas as pd
import time
import numpy as np
import random
import os
import sys
import codecs
import jieba
import pkuseg
import math
import pickle
import gensim
import stanfordnlp as stanfordnlp
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.models.word2vec import LineSentence
from sklearn.manifold import TSNE
import re, sys, codecs
import hanziconv
from hanziconv import HanziConv
import pytest
import re
import nltk
from nltk.tree import Tree
import matplotlib.pyplot as plt
%matplotlib inline

from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
from nltk.tokenize.stanford_segmenter import StanfordSegmenter
import stanfordcorenlp
import stanfordnlp
from stanfordcorenlp import StanfordCoreNLP

from functools import wraps
from collections import defaultdict
from collections import Counter
from functools import lru_cache

### 数据库读取数据（但数据存在截断，未使用其数据）

In [3]:
file_name = "/Users/lingrowzhang/Documents/GitHub/Artificial-Intelligence-for-NLP/"

In [3]:
headname = ["id","author","source","content","feature","title","url"]
news_chinese = pd.read_csv(file_name + "date/news_chinese.csv")
news_chinese.columns = headname

In [8]:
news_chinese.head(2)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm


In [25]:
data_content = list(news_chinese["content"])
data_content[6]

'虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是今年 3 月份官方突然宣布回归中国，预示着很快就有夏普新机在中国登场了。那么，第一款夏普手机什么时候登陆中国呢？又会是怎么样的手机呢？\r\n近日，一款型号为 FS8016 的夏普神秘新机悄然出现在 GeekBench 的跑分库上。从其中相关信息了解到，这款机子并非旗舰定位，所搭载的是高通骁龙 660 处理器，配备有 4GB 的内存。骁龙 660 是高通今年最受瞩目的芯片之一，采用 14 纳米工艺，八个 Kryo 260 核心设计，集成 Adreno 512 GPU 和 X12 LTE 调制解调器。\r\n当前市面上只有一款机子采用了骁龙 660 处理器，那就是已经上市销售的 OPPO R11。骁龙 660 尽管并非旗舰芯片，但在多核新能上比去年骁龙 820 强，单核改进也很明显，所以放在今年仍可以让很多手机变成高端机。不过，由于 OPPO 与高通签署了排他性协议，可以独占两三个月时间。\r\n考虑到夏普既然开始测试新机了，说明只要等独占时期一过，夏普就能发布骁龙 660 新品了。按照之前被曝光的渲染图了解，夏普的新机核心竞争优势还是全面屏，因为从 2013 年推出全球首款全面屏手机 EDGEST 302SH 至今，夏普手机推出了多达 28 款的全面屏手机。\r\n在 5 月份的媒体沟通会上，惠普罗忠生表示：“我敢打赌，12 个月之后，在座的各位手机都会换掉。因为全面屏时代的到来，我们怀揣的手机都将成为传统手机。”\r\n'

In [69]:
%%time
jieba.cut(data_content[6])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


<generator object Tokenizer.cut at 0x1bd2cb63b8>

In [70]:
%%time
pkuseg.pkuseg().cut(data_content[6])

CPU times: user 1.56 s, sys: 360 ms, total: 1.92 s
Wall time: 1.92 s


['虽然',
 '至今',
 '夏普',
 '智能',
 '手机',
 '在',
 '市场',
 '上',
 '无法',
 '排',
 '得',
 '上',
 '号',
 '，',
 '已经',
 '完全',
 '没落',
 '，',
 '并',
 '于',
 '2013',
 '年',
 '退出',
 '中国',
 '市场',
 '，',
 '但是',
 '今年',
 '3',
 '月份',
 '官方',
 '突然',
 '宣布',
 '回归',
 '中国',
 '，',
 '预示',
 '着',
 '很快',
 '就',
 '有',
 '夏普',
 '新机',
 '在',
 '中国',
 '登场',
 '了',
 '。',
 '那么',
 '，',
 '第一',
 '款',
 '夏普',
 '手机',
 '什么',
 '时候',
 '登陆',
 '中国',
 '呢',
 '？',
 '又',
 '会',
 '是',
 '怎么样',
 '的',
 '手机',
 '呢',
 '？',
 '近日',
 '，',
 '一',
 '款',
 '型号',
 '为',
 'FS8016',
 '的',
 '夏普',
 '神秘',
 '新机',
 '悄然',
 '出现',
 '在',
 'GeekBench',
 '的',
 '跑分库',
 '上',
 '。',
 '从',
 '其中',
 '相关',
 '信息',
 '了解',
 '到',
 '，',
 '这',
 '款',
 '机子',
 '并非',
 '旗舰',
 '定位',
 '，',
 '所',
 '搭载',
 '的',
 '是',
 '高通',
 '骁龙',
 '660',
 '处理器',
 '，',
 '配备',
 '有',
 '4GB',
 '的',
 '内存',
 '。',
 '骁龙',
 '660',
 '是',
 '高通',
 '今年',
 '最',
 '受',
 '瞩目',
 '的',
 '芯片',
 '之一',
 '，',
 '采用',
 '14',
 '纳米',
 '工艺',
 '，',
 '八',
 '个',
 'Kryo',
 '260',
 '核心',
 '设计',
 '，',
 '集成',
 'Adreno',
 '512',
 'GPU',
 '和',
 'X12',
 'LTE',
 '调

In [6]:
# 提取其中有用数据
all_author = []
all_type = []

In [7]:
all_source = []
for the_tmp in list(news_chinese["source"]):
    for the_str in str(the_tmp).split('@'):
        if "http" not in the_str and the_str != "":
            if "第" in the_str:
                try:
                    the_str_i = the_str.index("第")
                except:
                    continue
                if the_str[:the_str_i] not in all_source:
                    all_source.append(the_str[:the_str_i])
            else:
                if the_str not in all_source:
                    all_source.append(the_str)
all_source    

['快科技',
 '新华社',
 '深圳大件事',
 '中国证券报?中证网',
 '威锋网',
 '荆楚网-楚天都市报',
 '证券时报网',
 '中国新闻网',
 '国际在线',
 '中国证券网',
 'solidot',
 '新华网',
 '泡泡网',
 '凤凰体育',
 '雷锋网',
 '中超球评',
 '中国经济周刊',
 '人民日报',
 '央广网',
 '体育君',
 '新浪体育',
 '广州日报',
 'TechWeb',
 '足球小志',
 '人民网',
 '时代财经',
 '新疆日报',
 '中关村在线',
 '网易科技',
 '封面新闻',
 '央视新闻',
 '澎湃新闻网',
 '中青在线',
 '央视新闻客户端',
 '千龙网',
 '新浪综合',
 '中新网',
 'GamerSky游民星空',
 '中国经济网',
 'cnBeta.COM',
 '热点体育',
 '枪炮篮球PiU',
 '中国网财经',
 '知客数码',
 '于斌',
 '霍寿喜',
 '网易科学人',
 '看看新闻KNEWS',
 '最后一公里',
 '伊吾零距离',
 '环球时报-环球网',
 '经济日报',
 '足球部落国际版',
 '参考消息网',
 '黑龙江日报',
 '解放军报',
 '环球网',
 '星城古龙',
 '凤凰网科技',
 '每日经济新闻',
 '新浪科技',
 '经济参考报',
 '三十年莱斯特城球',
 '中国科技网',
 '柳州新闻网',
 '新京报即时新闻',
 'CCTV-央视网',
 '新闻晨报',
 '北青网-北京青年报',
 'linkgou.com',
 '中华网投资',
 '环球科技',
 '北京旅游网',
 '检察日报',
 '海南日报',
 '南方周末',
 '中国台湾网',
 '南宁晚报',
 '时刻体育',
 '北京晨报',
 '凤凰娱乐',
 '球娱天下',
 '中国娱乐网',
 '云南网',
 '中国时尚在线',
 '足球喜剧之王',
 '南国都市报',
 '剥洋葱people',
 '安卓中国',
 '中国青年报',
 '中国网',
 '网络',
 '深读',
 '哎，听说',
 '阳郊网警',
 '燕赵晚报',
 '证券日报',
 '重案组37号',
 '郑州晚报',
 '华尔街见闻',
 'Wind资讯',


In [9]:
all_author = []
for the_tmp in list(news_chinese["source"]):
    the_str = str(the_tmp)
    if "http" not in the_str and the_str != "" and the_str not in all_source:
            all_source.append(the_str)

## 模型前的数据处理

In [71]:
def tradition2simple(line):
    # 将繁体转换成简体
    return HanziConv.toSimplified(line)
# pkuseg分词
def pkuseg_cut(sent):
    seg = pkuseg.pkuseg()
    return seg.cut(sent)
def cut(string):
    return list(jieba.cut(string))

In [49]:
# 读入停用词表
stop_symbols = pd.read_csv(file_name+"date/test/stop_symbols.csv", encoding="utf-8")

In [60]:
def get_word(string_list):
    the_str  = ""
    for string in string_list:
        for the_chat in string:
            if ('\u4e00' <= the_chat <= '\u9fa5'):
                the_str += the_chat
        the_str = the_str + " "
    return the_str
all_word = defaultdict(int)
def del_stop_word(string):
    the_string = ""
    for the_word in string.split(" "):
        if the_word in stop_symbols or the_word == "":
            continue
        else:
            all_word[the_word] += 1
            the_string += the_word + " "
    return the_string

In [94]:
def wiki_words(file_index):
    with open(file_name + "date/wiki_66/deal_" + str(file_index) + '.txt', 'r', encoding='UTF-8') as f, \
            open(file_name + "date/wiki_66/pkuseg_" + str(file_index) + '.txt', 'w', encoding='UTF-8') as pkuseg_path:
        lines = [''.join(line) for line in f.readlines() if
                 not line.startswith("<doc") and not line.startswith("</doc>") and not len(line.strip()) == 0]
        for i, line in enumerate(lines):
            get_word_str = get_word(cut(line))
            pkuseg_path.write(get_word_str + '\n') 
            # del_stop_word_str = del_stop_word(get_word_str)
    return True

In [95]:
for path,_,files in os.walk(file_name+ "date/wiki_66/AA/"):
    for the_filename in files:
        if the_filename ==  '.DS_Store':
            continue
        wiki_words(the_filename[len(the_filename)-2:])
        print(the_filename)

wiki_11
wiki_16
wiki_18
wiki_19
wiki_17
wiki_10
wiki_03
wiki_04
wiki_05
wiki_02
wiki_15
wiki_12
wiki_13
wiki_14
wiki_09
wiki_07
wiki_00
wiki_01
wiki_06
wiki_08


In [ ]:
# 从维基百科取一整个词条为一行

In [ ]:
def try_readlines_the(file_index):
    with open(file_name + 'date/wiki_66/AA/wiki_' + file_index, 'r', encoding='UTF-8') as f:
        the_data = []
        the_result = []
        the_data_start  = 0
        for line in f.readlines():
            if line.startswith("<doc") and not len(line.strip()) == 0 and the_data_start == 0:
                the_data = []
                the_data_start = 1
            elif line.startswith("</doc") and the_data_start == 2:
                if the_data != '':
                    the_result.append(the_data)
                the_data_start = 0
            elif line in  ['', '\n']:
                continue
            else:
                line = line.strip(" ")
                line = line.strip("\n")
                the_data.append(tradition2simple(line))
                the_data_start = 2
    tmp_table = pd.DataFrame()
    tmp_table["data"] = the_result
    tmp_table.to_csv(file_name + "date/the_wiki/the_wiki_" + file_index + '.csv', encoding='UTF-8')
    return True
for path,_,files in os.walk(file_name + "date/wiki_66/AA/"):
    for the_file in files:
        if "wiki" in str(the_file):
            try_readlines_the(str(the_file[len(the_file)-2:]))
            print(the_file)
print("把一条数据整合成一行")

## 训练一个模型wiki

In [274]:
i=0
for path,_,files in os.walk(file_name + "date/wiki_66/"):
    for the_file in files:
        if "wiki" in str(the_file):
            with open(file_name + 'date/wiki_66/pkuseg_' + the_file[len(the_file)-2:] + '.txt', 'r', encoding='utf-8') as wiki_news:
                sentences = LineSentence(wiki_news)
                if i == 0:
                    i+=1
                    model_wiki_200 = Word2Vec(sentences, size=200, min_count=4, window=5, workers=8)
                else:
                    model_wiki_200.build_vocab(sentences, update=True)  # 更新词汇表
                    model_wiki_200.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # epoch=iter语料库的迭代次数；（默认为5）  total_examples:句子数。
                print(the_file)

wiki_11


/Users/lingrowzhang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if sys.path[0] == '':


wiki_16
wiki_18
wiki_19
wiki_17
wiki_10
wiki_03
wiki_04
wiki_05
wiki_02
wiki_15
wiki_12
wiki_13
wiki_14
wiki_09
wiki_07
wiki_00
wiki_01
wiki_06
wiki_08


In [275]:
model_wiki_200.save(file_name + 'model/word2vec_wiki_200_458' + '.model')

In [273]:
i=0
for path,_,files in os.walk(file_name + "date/wiki_66/"):
    for the_file in files:
        if "wiki" in str(the_file):
            with open(file_name + 'date/wiki_66/pkuseg_' + the_file[len(the_file)-2:] + '.txt', 'r', encoding='utf-8') as wiki_news:
                sentences = LineSentence(wiki_news)
                if i == 0:
                    i+=1
                    model_wiki_500 = Word2Vec(sentences, size=500, min_count=2, window=7, workers=8)
                else:
                    model_wiki_500.build_vocab(sentences, update=True)  # 更新词汇表
                    model_wiki_500.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # epoch=iter语料库的迭代次数；（默认为5）  total_examples:句子数。
                print(the_file)

KeyboardInterrupt: 

In [138]:
model_wiki_500.save(file_name +'model/word2vec_wiki_500' + '.model')

## 训练一个wiki+新闻的模型

In [142]:
content = []
for the_content in data_content:
    content += HanziConv.toSimplified(str(the_content)).split('\r\n')
data_content = []
for the_content in content:
    data_content.append(' '.join(cut(the_content)))
tmp_data = pd.DataFrame(data = data_content)

In [143]:
tmp_data.to_csv(file_name + "date/content.csv", encoding='utf-8')

In [145]:
model_wiki_new_200 = Word2Vec.load(file_name +'model/word2vec_wiki_200.model')
with open(file_name + "date/content.csv", 'r', encoding='utf-8') as wiki_news:
    sentences = LineSentence(wiki_news)
    model_wiki_new_200.build_vocab(sentences, update=True)  # 更新词汇表
    model_wiki_new_200.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # epoch=iter语料库的迭代次数；（默认为5）  total_examples:句子数。
    model_wiki_new_200.save(file_name + 'model/word2vec_wiki_new_200'+'.model')

/Users/lingrowzhang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


In [146]:
model_wiki_new_500 = Word2Vec.load(file_name +'model/word2vec_wiki_500.model')
with open(file_name + "date/content.csv", 'r', encoding='utf-8') as wiki_news:
    sentences = LineSentence(wiki_news)
    model_wiki_new_500.build_vocab(sentences, update=True)  # 更新词汇表
    model_wiki_new_500.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # epoch=iter语料库的迭代次数；（默认为5）  total_examples:句子数。
    model_wiki_new_500.save(file_name + 'model/word2vec_wiki_new_500'+'.model')

/Users/lingrowzhang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


## 新闻模型

In [193]:
with open(file_name + "date/content.csv", 'r', encoding='utf-8') as wiki_news:
    sentences = LineSentence(wiki_news)
    model_the_new_200 = Word2Vec(sentences, size=200, min_count=2, window=7, workers=8)
    model_the_new_200.save(file_name + 'model/word2vec_news_200'+'.model')

In [194]:
with open(file_name + "date/content.csv", 'r', encoding='utf-8') as wiki_news:
    sentences = LineSentence(wiki_news)
    model_the_new_500 = Word2Vec(sentences, size=500, min_count=2, window=7, workers=8)
    model_the_new_500.save(file_name + 'model/word2vec_news_500'+'.model')

### 用字典数据训练一份模型

In [8]:
dir_myself = pd.read_csv("/Users/lingrowzhang/LinGrow/dir/常见音字和词_20190828.csv")
dir_myself.fillna("")
dir_myself

,Unnamed: 0,拼音,字母,音调,中文,常见的词组（不包含语义，这是常见字纠错）,描述字的语句,老中文,部首,笔画_y,解释,更多
0,4105.0,ā,a,1.0,阿,阿尔法,NaN,阿,阝,7.0,"阿〈助〉\n \n (汉语名词词头,盛行于魏晋以后)\n \n 加在亲属称呼前面有亲昵的意味...",阿 a、e 部首 阝 部首笔画 03 总笔画 07 阿1\nā\n〈助〉\n(1)\n(汉...
1,768.0,ā,a,1.0,啊,啊呀，他溜的好快；惊讶为难,NaN,啊,口,10.0,"啊 \n\n (形声。从口,阿声。本义叹词) 表示惊叹或赞颂。如啊!多么美丽的夜色;啊!祖国...",啊 a 部首 口 部首笔画 03 总笔画 10 啊\nah；eh；oh；o；\n啊1\nā...
2,673.0,ā,a,1.0,呵,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,624.0,ā,a,1.0,吖,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3216.0,ā,a,1.0,腌,腌肉,NaN,NaN,NaN,NaN,NaN,NaN
5,767.0,á,a,2.0,啊,啊，你说什么；疑惑反问,NaN,NaN,NaN,NaN,NaN,NaN
6,4107.0,ǎ,a,3.0,阿,阿妈,NaN,NaN,NaN,NaN,NaN,NaN
7,769.0,ǎ,a,3.0,啊,啊，可以这样吗；惊疑,NaN,NaN,NaN,NaN,NaN,NaN
8,4104.0,à,a,4.0,阿,阿哥,NaN,NaN,NaN,NaN,NaN,NaN
9,766.0,à,a,4.0,啊,啊，好吧，我才明白；醒悟答应,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dir_myself = dir_myself.fillna("")
dir_myself.head()

,Unnamed: 0,拼音,字母,音调,中文,常见的词组（不包含语义，这是常见字纠错）,描述字的语句,老中文,部首,笔画_y,解释,更多
0,4105,ā,a,1,阿,阿尔法,,阿,阝,7,"阿〈助〉\n \n (汉语名词词头,盛行于魏晋以后)\n \n 加在亲属称呼前面有亲昵的意味...",阿 a、e 部首 阝 部首笔画 03 总笔画 07 阿1\nā\n〈助〉\n(1)\n(汉...
1,768,ā,a,1,啊,啊呀，他溜的好快；惊讶为难,,啊,口,10,"啊 \n\n (形声。从口,阿声。本义叹词) 表示惊叹或赞颂。如啊!多么美丽的夜色;啊!祖国...",啊 a 部首 口 部首笔画 03 总笔画 10 啊\nah；eh；oh；o；\n啊1\nā...
2,673,ā,a,1,呵,,,,,,,
3,624,ā,a,1,吖,,,,,,,
4,3216,ā,a,1,腌,腌肉,,,,,,


In [10]:
def tradition2simple(line):
    # 将繁体转换成简体
    return HanziConv.toSimplified(line)
# pkuseg分词
def pkuseg_cut(sent):
    seg = pkuseg.pkuseg()
    return seg.cut(sent)
def cut(string):
    return list(jieba.cut(string))
def get_word(string_list):
    result_str = ""
    the_str  = ""
    i = 0
    for string in string_list:
        if string in ['',' ','\n']:
            continue
        else:
            for the_chat in string:
                if ('\u4e00' <= the_chat <= '\u9fa5'):
                    the_str += the_chat
            if the_str in ['',' ','\n']:
                continue
            else:
                result_str += the_str + " "
                the_str = ""
    return result_str
all_word = defaultdict(int)

In [11]:
# 把后面2段拼接起来
all_text_dir = []
for dir_myself_i, the_data_1 in enumerate(list(dir_myself["解释"])):
    if the_data_1 != "":
        the_data_11 = get_word(cut(tradition2simple(str(the_data_1))))
        all_text_dir.append(the_data_11)
    the_data_2 = dir_myself.iloc[dir_myself_i]["更多"]
    if the_data_2 != "":
        the_data_22 = get_word(pkuseg_cut(tradition2simple(str(the_data_2))))
        all_text_dir.append(the_data_22)
    if the_data_11 =="":
        print(the_data_1)
    if the_data_22 =="":
        print(the_data_2)
all_text_dir

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/04/ww1l0rx91jjf64l2y45_44d80000gn/T/jieba.cache
Loading model cost 0.527 seconds.
Prefix dict has been built succesfully.


['阿 助 汉语 名词 词头 盛行于 魏晋 以后 加 在 亲属 称 唿 前面 有 亲昵 的 意味 阿姊闻 妹来 乐府诗 集 木兰 诗 府 吏得闻 之 堂上 启阿母 玉台新 咏 古诗 为 焦仲卿 妻作 又 如 阿爷 又 作 阿耶 称 父亲 阿翁 称 祖父 称 父亲 妇女 称 丈夫 之 父 阿嫂 对 朋友 之妻 的 称 唿 阿叔 对 丈夫 之弟 的 称 唿 阿家 又称 阿姑 妇人 称 丈夫 的 母亲 阿爹 称 父亲 也 尊称 长者 阿父 称 父亲 也 称 伯伯 叔叔 或作 伯叔 自称 阿伯 尊称 年长 的 男子 加 于 某些 人 的 姓 名 小名 排行 前 用作 称 唿 往往 带有 一定 的 感情 色彩 或 尊卑 关系 阿女 含泪 答 玉台新 咏 古诗 为 焦仲卿 妻作 又 如 阿咸 侄子 晋朝 阮籍 的 侄儿 阮咸 有 才 后来 遂 用来 称 侄子 阿连 弟弟 南朝 宋 谢灵运 的 族弟 惠连 很 有 才 人们 随 谢灵运 称之为 阿连 阿杜 原指周 盘龙 的 小 妾 杜氏 后 为 妾 的 别称 阿娇 汉武帝 陈皇后 的 小名 泛指 美丽 的 女子 阿蒙 三国 时 吴国 名将 吕蒙 阿瞒 三国 曹操 的 小字 阿 助词 用 在 称 唿前 姨 哥 张 阿昌族 我国 少数民族 之 一 阿 凹曲 处山 迎合 偏袒 谀 又 见 阿弥陀佛 梵文 的 音译 佛经 中 西方 极乐世界 的 教主 佛教 寺庙 中常 与 释迦 药师佛 并供 佛教徒 以 之 作为 口头 念诵 的 佛号 表示 虔诚 信仰 又 意译 作 无量 寿佛 无量 光佛 阿房宫 秦代 着 名 宫殿 遗址 在 西安市 西 阿房 村 始建 于 前 年 规模 极为 宏大 东西 五里 南北 千步 全部 工程 至 秦亡 时 犹未 完成 故未 正式 命名 时人用 其 前殿 所在地 名而 称之为 阿房宫 秦亡 后 被 项羽 焚毁 阿其所好 无原则 地 迎合 别人 的 爱好 阿谀 为 迎合 别人 说好听 的话 阿 叹词 表示 惊讶 阿 方言 副词 犹言 可 是否 用 在 问句 中 加强 语气 阿 语气词 用 在 句首 或句 中 无义 语气词 用 在 句末 表示 肯定 嘱咐 乞请 等 语气 语气词 作 啊 用 在 句 中 稍作 停顿 让 人 注意 下面 的话 ',
 '阿 部首 阝 部首 笔画 总笔画 阿 助 汉语 名词 词头 盛行 于

In [12]:
all_text_dir_table = pd.DataFrame(data = all_text_dir[1:])
all_text_dir_table.to_csv(file_name + "date/all_text_dir_table_pkuseg.csv")

In [266]:
with open(file_name + "date/all_text_dir_table.csv", 'r', encoding='utf-8') as all_text_dir_table:
    sentences = LineSentence(all_text_dir_table)
    model_the_dir_500 = Word2Vec(sentences, size=500, min_count=2, window=7, workers=8)
    model_the_dir_500.save(file_name + 'model/word2vec_dir_500'+'.model')

## 输出同义词

In [267]:
word = "说"

In [268]:
model_the_dir_500.wv.most_similar(word, topn=100)

[('谁', 0.9247288703918457),
 ('否', 0.9148572683334351),
 ('浪迹天涯', 0.9061311483383179),
 ('如此', 0.9018069505691528),
 ('还', 0.9015594720840454),
 ('断线风筝', 0.8984947204589844),
 ('却', 0.8971928954124451),
 ('只待', 0.8909785747528076),
 ('才', 0.8907827138900757),
 ('听', 0.8887753486633301),
 ('其珠', 0.8870226144790649),
 ('理所当然', 0.8853921294212341),
 ('可愿', 0.8851205706596375),
 ('转达', 0.8836381435394287),
 ('拉不下脸', 0.882657527923584),
 ('不能', 0.8799852728843689),
 ('好', 0.8768484592437744),
 ('祸从口出', 0.8762805461883545),
 ('认不出', 0.8761440515518188),
 ('得', 0.8759031891822815),
 ('利市三倍', 0.8755316734313965),
 ('晓得', 0.8750035762786865),
 ('一扣', 0.8746521472930908),
 ('我', 0.8741227984428406),
 ('虽说', 0.8712825775146484),
 ('真像', 0.8695696592330933),
 ('罢了', 0.8693056106567383),
 ('似说', 0.8682656288146973),
 ('不成话', 0.8671646118164062),
 ('浑浑噩噩', 0.8665043711662292),
 ('闻所未闻', 0.8664543628692627),
 ('如汝们', 0.8653323650360107),
 ('坏话', 0.8648208975791931),
 ('便', 0.8632076978683472),
 ('惊恐万

In [196]:
similar_speak = pd.DataFrame()

In [206]:
similar_speak_word = []
similar_speak_word_d = defaultdict(int)
similar_speak_word_n = defaultdict(int)
similar_speak_word_n_true = defaultdict(float)
def get_word(data):
    for the_data in data:
        if the_data not in similar_speak_word:
            similar_speak_word.append(the_data[0])
        similar_speak_word_d[the_data[0]] += 1
        similar_speak_word_n[the_data[0]] += the_data[1]
        similar_speak_word_n_true[the_data[0]] = similar_speak_word_n[the_data[0]]/similar_speak_word_d[the_data[0]]
            
get_word(model_wiki_200.wv.most_similar(word, topn=2000))
get_word(model_wiki_500.wv.most_similar(word, topn=2000))
get_word(model_wiki_new_200.wv.most_similar(word, topn=2000))
get_word(model_wiki_new_500.wv.most_similar(word, topn=2000))
get_word(model_the_new_200.wv.most_similar(word, topn=2000))
get_word(model_the_new_500.wv.most_similar(word, topn=2000))

In [207]:
similar_speak = pd.DataFrame(list(similar_speak_word_n_true))
similar_speak.to_csv(file_name + "date/similar_speak.csv", encoding='gbk')

In [205]:
similar_speak

,0
0,说道
1,声称
2,断言
3,指出
4,却说
5,中说
6,要说
7,确信
8,问道
9,认为


In [192]:
similar_speak_word_n_true

defaultdict(float,
            {'说道': 0.6825544635454813,
             '声称': 0.5525371183951696,
             '断言': 0.49669212102890015,
             '指出': 0.5077614039182663,
             '却说': 0.5436784823735555,
             '中说': 0.5202735116084417,
             '要说': 0.47783979028463364,
             '确信': 0.5124891102313995,
             '问道': 0.4911787584424019,
             '认为': 0.5329575488964716,
             '直言': 0.5535633563995361,
             '问': 0.5376899838447571,
             '告诉': 0.5539070417483648,
             '所说': 0.49447604417800906,
             '写道': 0.5546577970186869,
             '觉得': 0.4742392674088478,
             '讲': 0.510724330941836,
             '时说': 0.5865466992060343,
             '难道': 0.45076285004615785,
             '表示': 0.6067271530628204,
             '吗': 0.434654600918293,
             '还称': 0.44926614314317703,
             '告诫': 0.42676566541194916,
             '反问': 0.5182308157285055,
             '为什么': 0.5085330704847971,
    

In [185]:
similar_speak_word_n

defaultdict(int,
            {'说道': 4.095326781272888,
             '声称': 3.3152227103710175,
             '断言': 1.9867684841156006,
             '指出': 2.031045615673065,
             '却说': 3.262070894241333,
             '中说': 3.12164106965065,
             '要说': 1.9113591611385345,
             '确信': 2.049956440925598,
             '问道': 1.9647150337696075,
             '认为': 3.19774529337883,
             '直言': 3.321380138397217,
             '问': 3.2261399030685425,
             '告诉': 3.3234422504901886,
             '所说': 2.472380220890045,
             '写道': 3.3279467821121216,
             '觉得': 1.8969570696353912,
             '讲': 3.0643459856510162,
             '时说': 3.519280195236206,
             '难道': 2.253814250230789,
             '表示': 3.6403629183769226,
             '吗': 1.738618403673172,
             '还称': 1.7970645725727081,
             '告诫': 1.7070626616477966,
             '反问': 3.1093848943710327,
             '为什么': 3.051198422908783,
             '地说': 3.301

In [183]:
similar_speak_word_d

defaultdict(int,
            {'说道': 6,
             '声称': 6,
             '断言': 4,
             '指出': 4,
             '却说': 6,
             '中说': 6,
             '要说': 4,
             '确信': 4,
             '问道': 4,
             '认为': 6,
             '直言': 6,
             '问': 6,
             '告诉': 6,
             '所说': 5,
             '写道': 6,
             '觉得': 4,
             '讲': 6,
             '时说': 6,
             '难道': 5,
             '表示': 6,
             '吗': 4,
             '还称': 4,
             '告诫': 4,
             '反问': 6,
             '为什么': 6,
             '地说': 6,
             '答道': 6,
             '坚称': 4,
             '转述': 4,
             '宣称': 6,
             '提到': 6,
             '说起': 6,
             '暗示': 4,
             '相信': 5,
             '断定': 4,
             '看来': 6,
             '撒谎': 4,
             '说出': 4,
             '也许': 4,
             '从来不': 4,
             '明白': 6,
             '明言': 4,
             '回答': 6,
             '所言': 3,
             '以为

In [151]:
model_wiki_500.wv.most_similar(word, topn=500)

[('说道', 0.6133195757865906),
 ('问道', 0.5193837881088257),
 ('中说', 0.49918419122695923),
 ('却说', 0.4931553900241852),
 ('时说', 0.49296295642852783),
 ('要说', 0.4856141209602356),
 ('答道', 0.47784125804901123),
 ('地说', 0.4775414764881134),
 ('讲', 0.4655785858631134),
 ('断言', 0.4618067145347595),
 ('告诉', 0.4575996994972229),
 ('指出', 0.45684897899627686),
 ('所说', 0.45508110523223877),
 ('声称', 0.45388123393058777),
 ('确信', 0.4536489248275757),
 ('告诫', 0.4513098895549774),
 ('直言', 0.44700050354003906),
 ('撒谎', 0.4442172944545746),
 ('认为', 0.4414199888706207),
 ('说出', 0.4386502504348755),
 ('答', 0.4366823434829712),
 ('问', 0.4342387020587921),
 ('写道', 0.4240516424179077),
 ('反问', 0.4235619902610779),
 ('提到', 0.4206998348236084),
 ('还称', 0.4203680753707886),
 ('表示', 0.41984933614730835),
 ('责怪', 0.4177764654159546),
 ('驳斥', 0.4147683382034302),
 ('说起', 0.4137166142463684),
 ('难道', 0.4135563373565674),
 ('说谎', 0.41326284408569336),
 ('转述', 0.4104289412498474),
 ('随便', 0.4091324210166931),
 ('骂', 0

In [152]:
model_wiki_new_200.wv.most_similar(word, topn=500)

[('表示', 0.6663826704025269),
 ('说道', 0.6521764993667603),
 ('告诉', 0.6270490884780884),
 ('声称', 0.6148521304130554),
 ('称', 0.5947339534759521),
 ('确信', 0.5737417936325073),
 ('认为', 0.5724910497665405),
 ('觉得', 0.5657881498336792),
 ('指出', 0.5653408765792847),
 ('坦言', 0.5606881976127625),
 ('时说', 0.5412628054618835),
 ('宣称', 0.5379183292388916),
 ('断言', 0.528239905834198),
 ('问', 0.5110002756118774),
 ('所说', 0.5090852975845337),
 ('相信', 0.5075632929801941),
 ('写道', 0.5047348141670227),
 ('坚称', 0.4998898506164551),
 ('断定', 0.49548009037971497),
 ('看来', 0.4950844645500183),
 ('想到', 0.4935811161994934),
 ('为什么', 0.4900552034378052),
 ('地说', 0.4830406904220581),
 ('并不认为', 0.48091748356819153),
 ('透露', 0.4808551073074341),
 ('也许', 0.48008209466934204),
 ('怎么样', 0.4774121046066284),
 ('还称', 0.4768950939178467),
 ('喊道', 0.4745498299598694),
 ('问道', 0.470481812953949),
 ('他称', 0.4695565700531006),
 ('吗', 0.46784594655036926),
 ('反问', 0.46699288487434387),
 ('说出', 0.4653882086277008),
 ('讲', 0.4

In [156]:
model_wiki_new_500.wv.most_similar(word, topn=500)

[('说道', 0.5185390710830688),
 ('表示', 0.517458438873291),
 ('声称', 0.4354862570762634),
 ('确信', 0.43025583028793335),
 ('认为', 0.4294358491897583),
 ('告诉', 0.4262959659099579),
 ('称', 0.424970805644989),
 ('时说', 0.4197975993156433),
 ('地说', 0.4060603678226471),
 ('指出', 0.4048934578895569),
 ('坦言', 0.4015926718711853),
 ('问道', 0.3960777819156647),
 ('讲', 0.39483892917633057),
 ('宣称', 0.3914956748485565),
 ('却说', 0.3852420151233673),
 ('断言', 0.3847428560256958),
 ('辩称', 0.3762032091617584),
 ('问', 0.37554600834846497),
 ('觉得', 0.37486428022384644),
 ('中说', 0.37228864431381226),
 ('要说', 0.37129607796669006),
 ('所说', 0.36862635612487793),
 ('相信', 0.3662062883377075),
 ('坚称', 0.3580445349216461),
 ('提到', 0.3560861647129059),
 ('答道', 0.35535556077957153),
 ('反问', 0.3544364273548126),
 ('明说', 0.35178953409194946),
 ('断定', 0.3505241870880127),
 ('说出', 0.3504384160041809),
 ('为什么', 0.34946519136428833),
 ('坚信', 0.3485092520713806),
 ('还称', 0.34814056754112244),
 ('并不认为', 0.3462097644805908),
 ('记得

In [157]:
model_the_new_200.wv.most_similar(word, topn=500)

[('说道', 0.8099209070205688),
 ('大有裨益', 0.7758430242538452),
 ('之词', 0.7604333162307739),
 ('闭着', 0.7535355687141418),
 ('时说', 0.7519122362136841),
 ('写道', 0.7428866624832153),
 ('直言', 0.7428038120269775),
 ('表示', 0.7419215440750122),
 ('坦言', 0.7410591244697571),
 ('佐伊说', 0.7340067625045776),
 ('动情', 0.7335999608039856),
 ('赞美', 0.7314423322677612),
 ('提到', 0.7255179286003113),
 ('颐武', 0.7170578241348267),
 ('那昕', 0.7150828838348389),
 ('宋爱萍', 0.7125948667526245),
 ('道', 0.7111477255821228),
 ('孙峰', 0.708681583404541),
 ('潘说', 0.7069588303565979),
 ('高兴', 0.7040319442749023),
 ('地说', 0.7006975412368774),
 ('时称', 0.7001261711120605),
 ('参政', 0.6990324854850769),
 ('志工', 0.6987667679786682),
 ('一扇门', 0.6979497671127319),
 ('老道', 0.6952937841415405),
 ('砸锅卖铁', 0.6944719552993774),
 ('森说', 0.6929974555969238),
 ('感叹', 0.6857984662055969),
 ('颇感', 0.679140567779541),
 ('拜登', 0.6779223084449768),
 ('回击', 0.6738912463188171),
 ('友敌', 0.6732828617095947),
 ('病魔', 0.6712350845336914),
 ('三怪', 0.

In [158]:
model_the_new_500.wv.most_similar(word, topn=500)

[('说道', 0.8068038821220398),
 ('大有裨益', 0.7810614109039307),
 ('闭着', 0.7727833986282349),
 ('之词', 0.7706788778305054),
 ('佐伊说', 0.7680401802062988),
 ('直言', 0.7636655569076538),
 ('时说', 0.752225399017334),
 ('写道', 0.7475259304046631),
 ('坦言', 0.7411260008811951),
 ('动情', 0.7411008477210999),
 ('赞美', 0.739561915397644),
 ('表示', 0.7391452789306641),
 ('提到', 0.7362246513366699),
 ('那昕', 0.7283912897109985),
 ('道', 0.7238565683364868),
 ('宋爱萍', 0.7233502864837646),
 ('颐武', 0.7224763631820679),
 ('孙峰', 0.7166067957878113),
 ('高兴', 0.7082021236419678),
 ('母校', 0.7066418528556824),
 ('志工', 0.7041601538658142),
 ('时称', 0.704060435295105),
 ('答道', 0.700802743434906),
 ('潘说', 0.7002707123756409),
 ('回击', 0.6987623572349548),
 ('感叹', 0.6968029141426086),
 ('参政', 0.6943960189819336),
 ('友敌', 0.6940077543258667),
 ('病魔', 0.6937202215194702),
 ('颇感', 0.6935951709747314),
 ('地说', 0.6925234794616699),
 ('老道', 0.6922168135643005),
 ('邹巍', 0.6919638514518738),
 ('不及格', 0.6898559331893921),
 ('迷自', 0.6894